In [6]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import os
import dask.dataframe as dd

In [2]:
path = '../data/processed/'

for f in os.listdir(os.path.join(path, 'labels')):
    df = pd.read_csv(os.path.join(path, 'labels', f))
    print(f)
    print(df.value_counts(), df.nunique(), '\n\n')

primary_labels_neighbors_15_components_50_clust_size_250.csv
# label
 3.0       42958
-1.0       22291
 20.0      17267
 17.0      15723
 6.0       13646
 9.0       13583
 19.0      12511
 12.0      10323
 15.0       9060
 18.0       7749
 7.0        6224
 16.0       4528
 1.0        3261
 0.0        2539
 11.0       2070
 4.0        1675
 14.0       1242
 8.0        1140
 5.0         622
 10.0        446
 13.0        279
 2.0         272
dtype: int64 # label    22
dtype: int64 


primary_labels_neighbors_500_components_50_clust_size_250.csv
# label
1.0        183632
0.0          5777
dtype: int64 # label    2
dtype: int64 


primary_labels_pca_components_100_clust_size_50.csv
# label
-1.0       180450
 0.0         8303
 1.0          656
dtype: int64 # label    3
dtype: int64 


primary_labels_pca_components_50_clust_size_50.csv
# label
-1.0       184993
 1.0         4362
 0.0           54
dtype: int64 # label    3
dtype: int64 


primary_labels_neighbors_500_components_50_clust_size_5

Ok, this clearly isn't working and I'm not sure why. Let's try using a DataLoader to get each row (line) of interest and then calculate our sample statistics based off of that 

In [9]:
import dask.dataframe as dd

labels = pd.read_csv('../data/processed/labels/primary_labels_neighbors_50_components_50_clust_size_250.csv')
cols = dd.read_csv('../data/processed/primary.csv').columns

In [4]:
cluster_dict = {}
to_remove = []

for c in cols:
    if c.startswith('RP') or c.startswith('MT'):
        to_remove.append(c)
        
to_remove

['RP11-582J16.5',
 'RP11-148L24.1',
 'MTIF2',
 'MT2A',
 'RP11-96B5.3',
 'MTMR4',
 'RPS9',
 'RP11-434B12.1',
 'MTMR7',
 'RP11-81H14.2',
 'MTSS1L',
 'RPS15',
 'RP11-90K6.1',
 'RP4-625H18.2',
 'RP11-98G7.1',
 'RP11-798M19.6',
 'RP11-536G4.2',
 'RP11-110G21.1',
 'RP11-543C4.1',
 'RP13-39P12.3',
 'RP4-773N10.4',
 'RP11-706O15.1',
 'RP11-770J1.4',
 'RP11-290F24.6',
 'RP11-108M9.6',
 'RP11-490O6.2',
 'RP1-111C20.4',
 'MTHFD1',
 'RP11-834C11.4',
 'RP11-324I22.4',
 'RP5-884G6.2',
 'RP11-432J24.5',
 'RP11-521O16.1',
 'RP11-1094M14.11',
 'RP11-22N19.2',
 'RP11-977G19.11',
 'RP11-708J19.1',
 'RP11-60L3.1',
 'RP11-469H8.6',
 'RPS29',
 'RP11-126O1.5',
 'RP3-402G11.25',
 'RP5-890E16.5',
 'RP11-506M13.3',
 'RP11-298I3.4',
 'RP5-1177M21.1',
 'RP11-302B13.5',
 'RPL22L1',
 'MTR',
 'RP11-260M2.1',
 'RP11-61J19.5',
 'RP11-471B22.3',
 'RP11-315D16.4',
 'RP11-111M22.3',
 'RPLP0',
 'RP11-2E11.9',
 'RP11-491F9.5',
 'RP4-597N16.4',
 'RPS26',
 'RPL7A',
 'MTA1',
 'RP11-771K4.1',
 'RP11-279O9.4',
 'RP11-1085N6.5',

In [5]:
for clust in labels.value_counts().index:
    rows = labels[labels['# label'] == clust].index
    df = pd.read_csv('../data/processed/primary.csv', skiprows = lambda x: x not in rows, names=cols).drop(to_remove, axis=1)

    cluster_dict[clust] = df.sum().nlargest(1000).index 
    print(cluster_dict[clust][0: 10])

Index(['MALAT1', 'TMSB4X', 'TUBA1A', 'STMN1', 'TMSB10', 'PTMA', 'EEF1A1',
       'FTL', 'ACTB', 'TUBB2B'],
      dtype='object')


KeyboardInterrupt: 

In [ ]:
for c in cluster_dict:
    print(cluster_dict[c][0:100])

In [ ]:
cols = [c.lower() for c in df.columns]
# print(cols)
to_remove = []

for c in cols:
    if c.startswith('mt') or c.startswith('rp'):
        to_remove.append(c)
    
to_remove

In [ ]:
top_genes = dict()

for clust in labels['# label'].unique():
    print(f'Computing for cluster {clust}')
    to_keep = labels[labels['# label'] == clust].index
    clust_df = pd.read_csv('../data/processed/primary.csv', skiprows = lambda x: x not in to_keep, header=0)
    display(clust_df)
    top_genes[clust] = clust_df.sum().nlargest(n=1000).index
    print(top_genes[clust])

Now, let's visualize our clustering results 

In [ ]:
viz = pd.read_csv('../data/processed/umap/primary_reduction_neighbors_100_components_2.csv')
